In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import csv
import operator
import datetime
import os
import pickle
from collections import Counter

In [3]:
train_source = pd.read_parquet('./datagame-2023/label_train_source.parquet', engine='pyarrow')
test_source = pd.read_parquet('./datagame-2023/label_test_source.parquet', engine='pyarrow')
train_target = pd.read_parquet('./datagame-2023/label_train_target.parquet', engine='pyarrow')

In [4]:
meta_song = pd.read_parquet('./datagame-2023/meta_song.parquet', engine='pyarrow')
meta_song_composer = pd.read_parquet('./datagame-2023/meta_song_composer.parquet', engine='pyarrow')
meta_song_genre = pd.read_parquet('./datagame-2023/meta_song_genre.parquet', engine='pyarrow')
meta_song_lyricist = pd.read_parquet('./datagame-2023/meta_song_lyricist.parquet', engine='pyarrow')
meta_song_producer = pd.read_parquet('./datagame-2023/meta_song_producer.parquet', engine='pyarrow')
meta_song_titletext = pd.read_parquet('./datagame-2023/meta_song_titletext.parquet', engine='pyarrow')

In [5]:
train_source

,session_id,song_id,unix_played_at,play_status,login_type,listening_order
0,751,6027767fad949f3ca5e772df04924949,1659598909,1,7,1
1,751,041547bddb0a3e730f32db84c65868ca,1659599182,1,7,2
2,751,041547bddb0a3e730f32db84c65868ca,1659599184,0,7,3
3,751,041547bddb0a3e730f32db84c65868ca,1659599185,0,7,4
4,751,8b32f88104ecf859be934d9b45f30cd1,1659599218,1,7,5
...,...,...,...,...,...,...
11445175,458622,3ba8c1e455e27172edec64deff3a8b15,1662339887,2,7,16
11445176,458622,3ba8c1e455e27172edec64deff3a8b15,1662340090,2,7,17
11445177,458622,3ba8c1e455e27172edec64deff3a8b15,1662340294,2,7,18
11445178,458622,3ba8c1e455e27172edec64deff3a8b15,1662340497,2,7,19


In [92]:
meta_song

,song_id,artist_id,song_length,album_id,language_id,album_month
0,10a46165bb84e056438e06c35763ee61,39.0,202.0,202.0,3.0,2000-01
1,ff025522d0f8e7198a75a4e03edce55c,133.0,278.0,9615.0,3.0,1995-01
2,7feec55b825a1c93d55b3ef9af9f9be5,4147.0,306.0,19017.0,3.0,2015-06
3,ed8b14c6d0fccd1a95f9140933f809f3,114.0,263.0,21059.0,3.0,1989-01
4,2ce5e96e0213a6dac83c54b003418726,112.0,194.0,22531.0,3.0,2003-01
...,...,...,...,...,...,...
1030707,2e317504e175c1c6d18fa47b21b1a7f5,NaN,NaN,NaN,NaN,None
1030708,60f94d53d47ff60130924a320345b8f1,NaN,NaN,NaN,NaN,None
1030709,2c7c7dd5713f93222429a5f05732de3b,NaN,NaN,NaN,NaN,None
1030710,04e7a4fa2804ff363cf985fb83ac5d82,NaN,NaN,NaN,NaN,None


In [6]:
#看每個item出現的次數
count_item = Counter(train_source['song_id'])
item_count = pd.DataFrame({"class":count_item.keys(),"count": count_item.values()})

#保留出現次數大於五次的item
item_count = item_count[item_count['count']>=5]
item_count = item_count.reset_index(level=None, drop=True)
item_count

,class,count
0,6027767fad949f3ca5e772df04924949,1875
1,041547bddb0a3e730f32db84c65868ca,93
2,e7efab54028017e35a35d1b1637e210c,1814
3,3f8e8cbe4b5d55f07ba4c7ddfab624b7,898
4,a97177f0f37a2bae91d8e67831949392,56
...,...,...
200033,2b220fc3cbad45ec475ab9ad5cd18c53,8
200034,9730759edd4726e6c5849da3dafaf98f,14
200035,4ef749a6cf9b3d5cece376661719a5f4,6
200036,b097b7bf687f97535bbdb85131f493a2,5


In [7]:
train_source = train_source[train_source['song_id'].isin(item_count['class'])]
test_source = test_source[test_source['song_id'].isin(item_count['class'])]
train_target = train_target[train_target['song_id'].isin(item_count['class'])]
train_source.reset_index(inplace=True, drop=True)
test_source.reset_index(inplace=True, drop=True)
train_target.reset_index(inplace=True, drop=True)

In [8]:
train_source

,session_id,song_id,unix_played_at,play_status,login_type,listening_order
0,751,6027767fad949f3ca5e772df04924949,1659598909,1,7,1
1,751,041547bddb0a3e730f32db84c65868ca,1659599182,1,7,2
2,751,041547bddb0a3e730f32db84c65868ca,1659599184,0,7,3
3,751,041547bddb0a3e730f32db84c65868ca,1659599185,0,7,4
4,751,041547bddb0a3e730f32db84c65868ca,1659599253,0,7,10
...,...,...,...,...,...,...
10682408,458622,3ba8c1e455e27172edec64deff3a8b15,1662339887,2,7,16
10682409,458622,3ba8c1e455e27172edec64deff3a8b15,1662340090,2,7,17
10682410,458622,3ba8c1e455e27172edec64deff3a8b15,1662340294,2,7,18
10682411,458622,3ba8c1e455e27172edec64deff3a8b15,1662340497,2,7,19


In [9]:
meta_song = meta_song[meta_song['song_id'].isin(item_count['class'])]
meta_song_composer = meta_song_composer[meta_song_composer['song_id'].isin(item_count['class'])]
meta_song_genre = meta_song_genre[meta_song_genre['song_id'].isin(item_count['class'])]
meta_song_lyricist = meta_song_lyricist[meta_song_lyricist['song_id'].isin(item_count['class'])]
meta_song_producer = meta_song_producer[meta_song_producer['song_id'].isin(item_count['class'])]
meta_song_titletext = meta_song_titletext[meta_song_titletext['song_id'].isin(item_count['class'])]

meta_song.reset_index(inplace=True, drop=True)
meta_song_composer.reset_index(inplace=True, drop=True)
meta_song_genre.reset_index(inplace=True, drop=True)
meta_song_lyricist.reset_index(inplace=True, drop=True)
meta_song_producer.reset_index(inplace=True, drop=True)
meta_song_titletext.reset_index(inplace=True, drop=True)

In [10]:
meta_song_genre

,song_id,genre_id
0,7366ba2acd92eb262191bdaedf249462,b856b6781d370a3645c6dde0c20b3597
1,0145eadf202d4c7305aec592f7b9a50b,bb3d7b04b67d5aeb5ab145bdd70750da
2,38f01112ca17de3c98a18afb70a6d592,43244ec4c30a0dad837d982892bc0c05
3,724abe8167847fbb5d90f8bed0351486,b856b6781d370a3645c6dde0c20b3597
4,4ae00754a16db3e212953cb3525aa1c2,43244ec4c30a0dad837d982892bc0c05
...,...,...
220196,0bafd37ef9486a402b05f2799c6c1dad,b856b6781d370a3645c6dde0c20b3597
220197,5bc4837bb5f1ac07ddd855816b93e8f8,b856b6781d370a3645c6dde0c20b3597
220198,9302ea84e01ba5ceb2e695663dd9c981,e1d5802bb4e0f6ab79b2b9f4a5ff924b
220199,e7909a30dd352edad562c5db2fc2135d,b856b6781d370a3645c6dde0c20b3597


In [11]:
song_id = LabelEncoder()
artist_id = LabelEncoder()
song_length = LabelEncoder()
album_id = LabelEncoder()
language_id = LabelEncoder()
album_month = LabelEncoder()
composer_id = LabelEncoder()
genre_id = LabelEncoder()
lyricist_id	 = LabelEncoder()
producer_id = LabelEncoder()
title_text_id = LabelEncoder()

In [12]:
song_id.fit(pd.concat([meta_song['song_id'], meta_song_composer['song_id'], meta_song_composer['song_id'], meta_song_genre['song_id'], meta_song_lyricist['song_id'], meta_song_producer['song_id'], meta_song_titletext['song_id']]))
artist_id.fit(meta_song['artist_id'])
song_length.fit(meta_song['song_length'])
album_id.fit(meta_song['album_id'])
language_id.fit(meta_song['language_id'])
album_month.fit(meta_song['album_month'])
composer_id.fit(meta_song_composer['composer_id'])
genre_id.fit(meta_song_genre['genre_id'])
lyricist_id.fit(meta_song_lyricist['lyricist_id'])
producer_id.fit(meta_song_producer['producer_id'])
title_text_id.fit(meta_song_titletext['title_text_id'])

meta_song['song_id'] = song_id.transform(meta_song['song_id'])+1
meta_song['artist_id'] = artist_id.transform(meta_song['artist_id'])+1
meta_song['song_length'] = song_length.transform(meta_song['song_length'])+1
meta_song['album_id'] = album_id.transform(meta_song['album_id'])+1
meta_song['language_id'] = language_id.transform(meta_song['language_id'])+1
meta_song['album_month'] = album_month.transform(meta_song['album_month'])+1

meta_song_composer['song_id'] = song_id.transform(meta_song_composer['song_id'])+1
meta_song_composer['composer_id'] = composer_id.transform(meta_song_composer['composer_id'])+1

meta_song_genre['genre_id'] = genre_id.transform(meta_song_genre['genre_id'])+1
meta_song_genre['song_id'] = song_id.transform(meta_song_genre['song_id'])+1

meta_song_lyricist['song_id'] = song_id.transform(meta_song_lyricist['song_id'])+1
meta_song_lyricist['lyricist_id'] = lyricist_id.transform(meta_song_lyricist['lyricist_id'])+1

meta_song_producer['song_id'] = song_id.transform(meta_song_producer['song_id'])+1
meta_song_producer['producer_id'] = producer_id.transform(meta_song_producer['producer_id'])+1

meta_song_titletext['song_id'] = song_id.transform(meta_song_titletext['song_id'])+1
meta_song_titletext['title_text_id'] = title_text_id.transform(meta_song_titletext['title_text_id'])+1

In [13]:
meta_song['artist_id'] = meta_song['artist_id']+max(meta_song['song_id'])
meta_song['song_length'] = meta_song['song_length']+max(meta_song['artist_id'])
meta_song['album_id'] = meta_song['album_id']+max(meta_song['song_length'])
meta_song['language_id'] = meta_song['language_id']+max(meta_song['album_id'])
meta_song['album_month'] = meta_song['album_month']+max(meta_song['language_id'])

meta_song_composer['composer_id'] = meta_song_composer['composer_id']+max(meta_song['album_month'])
meta_song_genre['genre_id'] = meta_song_genre['genre_id']+max(meta_song_composer['composer_id'])
meta_song_lyricist['lyricist_id'] = meta_song_lyricist['lyricist_id']+max(meta_song_genre['genre_id'])
meta_song_producer['producer_id'] = meta_song_producer['producer_id']+max(meta_song_lyricist['lyricist_id'])
meta_song_titletext['title_text_id'] = meta_song_titletext['title_text_id']+max(meta_song_producer['producer_id'])

In [14]:
max(meta_song_titletext['title_text_id'])

411563

In [15]:
NONE = max(meta_song['album_month'])
meta_song['artist_id'] = meta_song['artist_id'].replace(max(meta_song['artist_id']), NONE)
meta_song['song_length'] = meta_song['song_length'].replace(max(meta_song['song_length']), NONE)
meta_song['album_id'] = meta_song['album_id'].replace(max(meta_song['album_id']), NONE)
meta_song['language_id'] = meta_song['language_id'].replace(max(meta_song['language_id']), NONE)


In [16]:
max(meta_song['song_id'])

200038

In [18]:
doc_list = [meta_song, meta_song_composer, meta_song_genre, meta_song_lyricist, meta_song_producer, meta_song_titletext] 
with open('./datagame-2023/kg.txt', "w") as f:
    total_relation = 0
    for i in doc_list:
        for index, row in i.iterrows():
            now = 0
            song = row['song_id']
            for i, name in row.iteritems():
                if i == 'song_id':
                    continue
                elif name == NONE:
                    now += 1
                    continue
                line = str(song)+' '+str(total_relation + now)+' '+str(name)+'\n'
                f.write(line)
                now += 1
        total_relation += now
        print(total_relation)

5
6
7
8
9
10


In [19]:
train_target['song_id'] = song_id.transform(train_target['song_id'])+1
train_source['song_id'] = song_id.transform(train_source['song_id'])+1
test_source['song_id'] = song_id.transform(test_source['song_id'])+1

C:\Users\c8905\AppData\Local\Temp\ipykernel_19444\942974910.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_source['song_id'] = song_id.transform(train_source['song_id'])+1


In [22]:
sess_target = {}
ctr = 0
curid = -1
for index, row in train_target.iterrows():
    sessid = int(row['session_id'])
    if not curid == sessid:
        curid = sessid
        item = int(row['song_id'])
    else:
        continue
        
    if sessid in sess_target:
        continue
    else:
        sess_target[sessid] = item

In [23]:
sess_target

{307: 92132,
 1504: 41584,
 3505: 54385,
 4207: 64411,
 4974: 30511,
 5403: 147447,
 6277: 150658,
 7106: 126317,
 7312: 523,
 7333: 91413,
 8015: 93222,
 8403: 144180,
 8630: 85769,
 8707: 100329,
 10291: 123668,
 11085: 78530,
 11126: 194698,
 11430: 16700,
 11644: 60000,
 12162: 85203,
 13172: 141172,
 13490: 134123,
 13945: 179185,
 15676: 108097,
 15947: 102465,
 16806: 60531,
 18340: 194764,
 18737: 104302,
 20020: 95948,
 20171: 4557,
 20433: 77846,
 20887: 82067,
 21776: 33766,
 22860: 159116,
 23805: 179371,
 24972: 63991,
 25721: 111763,
 26614: 63461,
 29130: 92746,
 29366: 38617,
 29710: 4313,
 30553: 60544,
 30967: 12147,
 31894: 195454,
 34233: 149082,
 35042: 23308,
 35282: 42559,
 35895: 12254,
 36621: 28650,
 36680: 114194,
 37246: 33614,
 37782: 182110,
 38316: 55501,
 39206: 68491,
 39480: 42346,
 39626: 73319,
 39662: 102465,
 40342: 154713,
 40684: 110145,
 41385: 46976,
 41395: 23308,
 41474: 5321,
 41522: 171955,
 41961: 100586,
 42377: 150081,
 42762: 182025,
 4

In [24]:
sess_clicks = {}
sess_date = {}
ctr = 0
curid = -1
curdate = None
for index, row in train_source.iterrows():
    sessid = int(row['session_id'])
    if curdate and not curid == sessid:
        date = curdate
        sess_date[curid] = date
    curid = sessid
    item = int(row['song_id'])
    curdate = float(row['unix_played_at'])

    if sessid in sess_clicks:
        sess_clicks[sessid] += [item]
    else:
        sess_clicks[sessid] = [item]
    ctr += 1
date = float(row['unix_played_at'])
sess_date[curid] = date

In [25]:
dates = list(sess_date.items())

maxdate = max(train_source["unix_played_at"])
splitdate = 0

splitdate = maxdate - 86400 * 30   # the number of seconds for a day：86400

print('Splitting date', splitdate)  
tra_sess = list(filter(lambda x: int(x[1]) < splitdate, dates))
tes_sess = list(filter(lambda x: int(x[1]) > splitdate, dates))

tra_sess = sorted(tra_sess, key=operator.itemgetter(1))  # [(session_id, timestamp), (), ]
tes_sess = sorted(tes_sess, key=operator.itemgetter(1))  # [(session_id, timestamp), (), ]
print(len(tra_sess))  # 186670    # 7966257
print(len(tes_sess))  # 15979     # 15324
print(tra_sess[:3])
print(tes_sess[:3])
#print("-- Splitting train set and test set @ %ss" % datetime.datetime.now())

item_dict = {}


# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_ids = []
    train_seqs = []
    train_dates = []
    item_ctr = 1
    for s, date in tra_sess:
        seq = sess_clicks[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [i]
            else:
                outseq += [i]
                item_dict[i] = i
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_ids += [s]
        train_dates += [date]
        train_seqs += [outseq]
    print('item_ctr')
    print(item_ctr)     # 43098, 37484
    return train_ids, train_dates, train_seqs

# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_ids = []
    test_seqs = []
    test_dates = []
    for s, date in tes_sess:
        seq = sess_clicks[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [i]
        if len(outseq) < 2:
            continue
        test_ids += [s]
        test_dates += [date]
        test_seqs += [outseq]
    return test_ids, test_dates, test_seqs

tra_ids, tra_dates, tra_seqs = obtian_tra()
tes_ids, tes_dates, tes_seqs = obtian_tes()

def process_seqs(seqid, iseqs, idates):
    out_seqs = []
    out_dates = []
    labs = []
    ids = []
    for id, sessionID, seq, date in zip(range(len(iseqs)), seqid, iseqs, idates):
        for i in range(1, 2):
            if sessionID in sess_target:
                tar = sess_target[sessionID]
                labs += [tar]
                out_seqs += [seq[:-i]]
                out_dates += [date]
                ids += [sessionID]
    return out_seqs, out_dates, labs, ids

tr_seqs, tr_dates, tr_labs, tr_ids = process_seqs(tra_ids, tra_seqs, tra_dates)
te_seqs, te_dates, te_labs, te_ids = process_seqs(tes_ids, tes_seqs, tes_dates)
tra = (tr_seqs, tr_labs)
tes = (te_seqs, te_labs)
print(len(tr_seqs))
print(len(te_seqs))
print(tr_seqs[:3], tr_dates[:3], tr_labs[:3])
print(te_seqs[:3], te_dates[:3], te_labs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('avg length: ', all * 1.0/(len(tra_seqs) + len(tes_seqs)))

if not os.path.exists('KKBOX'):
    os.makedirs('KKBOX')
pickle.dump(tra, open('KKBOX/train.txt', 'wb'))
pickle.dump(tes, open('KKBOX/test.txt', 'wb'))
pickle.dump(tra_seqs, open('KKBOX/all_train_seq.txt', 'wb'))

print('Done.')


Splitting date 1664553437
391662
178862
[(273896, 1659225628.0), (43714, 1659225654.0), (84153, 1659225798.0)]
[(560494, 1664553454.0), (249084, 1664553481.0), (393650, 1664553485.0)]
item_ctr
195499
385625
174346
[[15525, 17365, 6245, 145480, 165890, 175921, 163565, 150166, 176667, 18582, 56024, 128862, 185074, 75821, 17836, 179802, 2798, 74482, 177880], [109059, 64436, 123675, 21555, 87925, 166468, 78207, 170508, 87117, 148304, 74007, 12737, 106388], [135067, 176955, 176955, 57460]] [1659225628.0, 1659225654.0, 1659225798.0] [185728, 25941, 135335]
[[47948, 98679, 63281, 114930, 142905, 125196, 23103, 107656, 167638, 150092, 42695, 101776, 18711, 115265, 165272, 32731], [156081, 156081, 156081, 156081, 156081, 156081, 156081, 156081, 156081, 156081, 156081, 156081, 156081, 156081, 156081, 156081, 156081, 156081, 156081], [183437, 183437, 183437, 183437, 183437, 183437, 183437, 183437, 183437, 183437, 183437, 183437, 183437, 183437, 183437, 183437, 183437]] [1664553454.0, 1664553481.0

# 將test data轉換為模型的輸入格式

In [26]:
sess_clicks = {}
sess_date = {}
ctr = 0
curid = -1
curdate = None
for index, row in test_source.iterrows():
    sessid = int(row['session_id'])
    if curdate and not curid == sessid:
        date = curdate
        sess_date[curid] = date
    curid = sessid
    item = int(row['song_id'])
    curdate = float(row['unix_played_at'])

    if sessid in sess_clicks:
        sess_clicks[sessid] += [item]
    else:
        sess_clicks[sessid] = [item]
    ctr += 1
date = float(row['unix_played_at'])
sess_date[curid] = date

item_dict = {}

def obtian_tes():
    test_ids = []
    test_seqs = []
    test_dates = []
    item_ctr = 1
    for s in sess_clicks.keys():
        seq = sess_clicks[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [i]
            else:
                outseq += [i]
                item_dict[i] = i
                item_ctr += 1
        test_ids += [s]
        test_seqs += [outseq]
    return test_ids, test_seqs
tes_ids, tes_seqs = obtian_tes()

def process_seqs(seqid, iseqs):
    out_seqs = []
    out_dates = []
    labs = []
    ids = []
    for id, sessionID, seq in zip(range(len(iseqs)), seqid, iseqs):
        for i in range(1, 2):
            out_seqs += [seq]
            labs += [0]
            ids += [sessionID]
    return out_seqs, labs, ids

te_seqs, te_labs, te_ids = process_seqs(tes_ids, tes_seqs)
tes = (te_seqs, te_labs)
pickle.dump(tes, open('./KKBOX/true_test.txt', 'wb'))

# 訓練完要把結果放過來轉換

In [ ]:
count_session = Counter(test_source['session_id'])
session_list = pd.DataFrame({"class":count_session.keys(),"count": count_session.values()})

In [ ]:
test = pd.read_parquet('./datagame-2023/label_test_source.parquet', engine='pyarrow')
count_test = Counter(test['session_id'])
test_list = pd.DataFrame({"class":count_test.keys(),"count": count_test.values()})
test_list['class']

In [ ]:
difference = list(set(test_list['class']).difference(set(session_list['class'])))
len(difference)

In [ ]:
prediction = pd.read_csv('./prediction.txt', sep=' ', names=['session_id', 'top1', 'top2', 'top3', 'top4', 'top5'])
prediction.drop_duplicates( keep='first', inplace=True)

In [ ]:
prediction = prediction.sort_values(by='session_id')
prediction

In [ ]:
prediction['session_id'] = list(session_list['class'])
prediction

In [ ]:
difference = pd.DataFrame(difference, columns=['session_id'])
prediction = pd.concat([prediction, difference], ignore_index=True).fillna(57959).astype(int)

In [ ]:
prediction['top1'] = song_id.inverse_transform(prediction['top1'])
prediction['top2'] = song_id.inverse_transform(prediction['top2'])
prediction['top3'] = song_id.inverse_transform(prediction['top3'])
prediction['top4'] = song_id.inverse_transform(prediction['top4'])
prediction['top5'] = song_id.inverse_transform(prediction['top5'])

In [ ]:
prediction.to_csv('./result.csv', index=False)